In [12]:
import time
import requests
from collections import OrderedDict

import tmdbsimple as tmdb # https://www.themoviedb.org/documentation/api
tmdb.API_KEY = ''
import imdb

from modules import IO, CountRequest

In [2]:
data_folder = 'data/'
cr = CountRequest()

In [3]:
tmdb_genres = tmdb.Genres().movie_list()
cr.add()
old_ids = []
tmdb_genres_list = []
for g in tmdb_genres['genres']:
    old_ids.append(g['id'])
    tmdb_genres_list.append(g['name'])
old_id2new_id = {_i:i for i, _i in enumerate(old_ids)}
IO(data_folder + 'tmdb_genres_list.pkl').to_pickle(tmdb_genres_list)

In [3]:
tmdb_genres_list = IO(data_folder + 'tmdb_genres_list.pkl').read_pickle()

print(len(tmdb_genres_list))
tmdb_genres_list

19


['Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western']

In [5]:
%%time

years = [y for y in range(1880, 2030)]
tmdb_all = []

for year in years:
    discover = tmdb.Discover().movie(year=year)
    cr.add()
    print(year, discover['total_pages'], discover['total_results'])
    for page in range(1, min(discover['total_pages']+1, 1001)):
        tmdb_all.extend(tmdb.Discover().movie(year=year, page=page)['results'])
        cr.add()

for m in tmdb_all:
    m['genre_ids'] = [old_id2new_id[i] for i in m['genre_ids']]

print(len(tmdb_all))
    
IO(data_folder + 'tmdb_all.pkl').to_pickle(tmdb_all)

1880 1 0
1881 1 1
1882 1 0
1883 1 1
1884 1 2
1885 1 0
1886 1 1
1887 1 1
1888 1 4
1889 1 2
1890 1 7
1891 1 6
1892 1 7
1893 1 1
1894 2 32
1895 3 48
1896 16 307
Sleep 10 seconds...
1897 14 278
1898 10 185
Sleep 10 seconds...
1899 10 195
1900 19 363
1901 11 204
Sleep 10 seconds...
1902 9 167
1903 12 240
1904 11 201
1905 9 179
Sleep 10 seconds...
1906 10 186
1907 9 172
1908 13 258
Sleep 10 seconds...
1909 14 262
1910 15 295
1911 19 378
Sleep 10 seconds...
1912 29 576
Sleep 10 seconds...
1913 24 479
1914 19 373
Sleep 10 seconds...
1915 21 405
1916 21 408
Sleep 10 seconds...
1917 21 411
1918 19 376
Sleep 10 seconds...
1919 20 382
1920 22 427
Sleep 10 seconds...
1921 20 400
Sleep 10 seconds...
1922 21 404
1923 21 417
Sleep 10 seconds...
1924 25 496
1925 28 558
Sleep 10 seconds...
1926 34 665
Sleep 10 seconds...
1927 35 687
Sleep 10 seconds...
1928 39 771
Sleep 10 seconds...
1929 41 818
Sleep 10 seconds...
1930 51 1018
Sleep 10 seconds...
1931 59 1164
Sleep 10 seconds...
Sleep 10 seconds...
193

Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
2012 661 13217
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
2013 742 14830
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
Sleep 10 seconds...
2014 794 15874
Sleep 10 second

In [4]:
tmdb_all = IO(data_folder + 'tmdb_all.pkl').read_pickle()
print(len(tmdb_all))
print(tmdb_all[10])

361622
{'vote_count': 2, 'id': 32406, 'video': False, 'vote_average': 4.5, 'title': 'Leisurely Pedestrians, Open Topped Buses and Hansom Cabs with Trotting Horses', 'popularity': 1.077166, 'poster_path': '/tYDotjkxDeCeIqP2UpI9oTWt06W.jpg', 'original_language': 'xx', 'original_title': 'Leisurely Pedestrians, Open Topped Buses and Hansom Cabs with Trotting Horses', 'genre_ids': [5], 'backdrop_path': None, 'adult': False, 'overview': 'Alleged silent black-and-white short film shot at Apsley Gate, Hyde Park, London. It was claimed to be the first motion picture until pre-dating footage shot by Louis Le Prince was discovered. It was never publicly shown and is now considered a lost film with no known surviving prints or stills.', 'release_date': '1889-01-14'}


In [9]:
data = tmdb_all

data = list(filter(lambda d:int(d['release_date'].split("-")[0])>=1980, data))
data = list(filter(lambda d:d['poster_path']!=None, data))
def should_keep(overview):
    if len(overview.split()) < 10:
        return False
    if overview == ".. \\ h /\r . n X y\r .. / c \\":
        return False
    return True
data = list(filter(lambda d:should_keep(d['overview']), data))
print(len(data))
data = data[::-1]

IO(data_folder + 'tmdb_processed.pkl').to_pickle(data)

183177


In [10]:
data = IO(data_folder + 'tmdb_processed.pkl').read_pickle()

In [11]:
data[0]

{'adult': False,
 'backdrop_path': '/19URihLRPW7r2sFSvjpFZDKfrAM.jpg',
 'genre_ids': [14, 2],
 'id': 465559,
 'original_language': 'ja',
 'original_title': '零世紀 メーテル',
 'overview': 'The story of Princess Maetel and how she came to start her timeless train trip on the Galaxy Express 999.',
 'popularity': 1.059256,
 'poster_path': '/oFbrPffgM94psqkdxiXMTlCvCtU.jpg',
 'release_date': '2026-01-01',
 'title': 'The Zero Century: Maetel',
 'video': False,
 'vote_average': 0,
 'vote_count': 0}

In [16]:
poster_folder = 'data/posters/'

In [20]:
%%time

def get_poster(moive_dict):
    url = 'http://image.tmdb.org/t/p/original' + moive_dict['poster_path']
    return requests.get(url).content

tmdb_posters = []

for i, movie in enumerate(data[:30000]):
    img = get_poster(movie)
    with open(poster_folder + str(i) + '.jpg', 'wb') as f:
        f.write(img)
    tmdb_posters.append(movie)
    print(i, movie['title'])
    time.sleep(0.3)
    
IO(data_folder + 'tmdb_posters.pkl').to_pickle(tmdb_posters)

0 The Zero Century: Maetel
1 Seed in the Sand
2 Knave
3 How Do You Guys Live?
4 Echo Planet
5 Q
6 Nimona
7 SpongeBob Squarepants 3
8 The Kingdom of the Two Sicilies
9 S.C.O.O.B.
10 Aleister Arcane
11 Charlatan
12 Astro Boy
13 Mahabharatha
14 G.I. Joe
15 Dawgtown
16 Mulan
17 Green Lantern Corps
18 Daddy's Girl
19 Barbie
20 Flashpoint
21 Guardians of the Galaxy Vol. 3
22 Ghostbusters
23 Stoner
24 Fate/stay night: Heaven's Feel III
25 Klaus
26 Angry Birds 2
27 I'm Thinking of Ending Things
28 The Painted Bird
29 Pet Sematary
30 Making a Killing
31 Random Acts of Violence
32 Aadujeevitham
33 Veeramadevi
34 Pray for Dawn
35 Naadaga Medai
36 The Kitchen
37 Excelsis
38 Chekka Chivantha Vaanam
39 The Bearer of Sorrows
40 Iron of the Conquistadors
41 Hazte Hombre
42 Seeds
43 Klonoa
44 Absolute Access
45 The Crow:Reborn
46 Divine
47 Everest
48 Kaaliyan
49 売春婦はアルファベットを書く
50 Crossfire
51 The Knight of Shadows: Between Yin and Yan
52 About Endlessness
53 Sleeping Dogs
54 Smith
55 Charlatan
56 Ahne

394 Careful How You Go
395 Daivame Kaithozham K.Kumar Akanam
396 Lazare
397 White Rabbit
398 McQueen
399 One More Shot
400 Victor Goodview
401 Shino Can't Say Her Own Name
402 Alastor
403 Lover
404 The Way Home
405 Agua Viva
406 Mud
407 911: Officer Down
408 Shanel
409 Happy Valentine's Day
410 The Girl with Two Faces
411 MexMan
412 Untouchable Love
413 The Queen of Fear
414 Saint Seiya: Knights of the Zodiac
415 The Boss Begins
416 Celestial Camel
417 Non è mai Passato
418 Crying on Command
419 A Blue Mouthed Face
420 Red Shoes & the 7 Dwarfs
421 Mother, May I Dance with Mary Jane's Fist?: A Lifetone Original Movie
422 Chooka
423 12 Noisy Shots
424 Chef Flynn
425 Haseena
426 Ghost in the Graveyard
427 Kailash
428 Europa
429 3 Gante 30 Dina 30 Second
430 Akicita: The Battle of Standing Rock
431 Egregor
432 The Real T Rex with Chris Packham
433 Ant-Babe
434 Left Behind and the Translation of God
435 Detroit Under S.T.R.E.S.S.
436 Moonfall
437 The Climb
438 Oxidan
439 Humble Politician N

781 Departure
782 You're Welcome
783 Loss
784 Our Blood Is Wine
785 Fraud Squad
786 End Game
787 Sofra Sırları
788 Eternal Winter
789 Beyond the Rye
790 Dark Buildings (A Crack in the Wall)
791 Luz
792 Symphony of a Sad Sea
793 Garbage
794 Going Fishing
795 Laurin
796 Gohin Baluchor
797 Bayi Gaib
798 Blood, Sweat and Lies
799 Shakedown
800 Hair Wolf
801 Rising Fear
802 New Town Utopia
803 Rust
804 Adriana's Pact
805 Love In The Wrong Places
806 Where the Summer Goes (Chapters on Youth)
807 A Man of Integrity
808 Busker
809 Rainbow Days
810 Gathering of Heroes: Legend of the Seven Swords
811 A Journey to the Fumigated Towns
812 Jane Fonda in Five Acts
813 Shikkari Shambhu
814 Cuban Food Stories
815 M/M
816 A Forest of Wool and Steel
817 Squaddies on the Frontline
818 Epilogue
819 E
820 The Heiresses
821 The 2018 American Rescue Dog Show
822 House Within The Night
823 Jahilya
824 Michael Palin: A Life on Screen
825 French Kisses
826 My Last Year as a Loser
827 Grass
828 Hello Apartment
8

1151 LAbyrinth
1152 Daha
1153 The BBQ
1154 Jai Simha
1155 The Heretic
1156 Polaroid
1157 Cool Cat The Kids Superhero
1158 Mau Nieto: Viviendo sobrio...desde el bar
1159 Fakkah Fuzz: Almost Banned
1160 Bucky and the Squirrels
1161 Up and Away
1162 Tremors: A Cold Day in Hell
1163 Christopher Robin
1164 Is the Order a Rabbit?? ~Dear My Sister~
1165 L'Animale
1166 An Elephant Sitting Still
1167 I Am A Star
1168 6 Athiyayam
1169 Silariang: Cinta Yang (Tak) Direstui
1170 Saul's 108th Story
1171 The October Flowers
1172 Ex-Shaman
1173 SuperFly
1174 Marie Curie
1175 The Two Bills
1176 Naa Peru Surya
1177 Wish
1178 Jamsil
1179 The Mimic
1180 Morning Show Mystery: Mortal Mishaps
1181 A Polar Year
1182 Super November
1183 Top Cat Begins
1184 Closeness
1185 Phantom Islands
1186 Oh Lucy!
1187 Zero to Heaven
1188 Veera
1189 Angel
1190 Isaac and the Volcano
1191 TRIO - The Hunt for the Holy Shrine
1192 Playing God
1193 Naragasooran
1194 Australien Skies 2: Contact Of Interest
1195 NJPW Honor Rising:

1509 Belle and Bamber
1510 Chronicle of a Farewell
1511 The Woman Who Left
1512 Grandpa's Great Escape
1513 Thy Kingdom Come
1514 Wunderland
1515 SECHSKIES Eighteen
1516 Break Night
1517 Delirium
1518 The Widowed Witch
1519 War of the Ants
1520 The Will to Fly
1521 Untitled Tenderness Project
1522 The Crimes That Bind
1523 Mirai of the Future
1524 Ang Pambansang Third Wheel
1525 Have a Nice Day
1526 Et le Diable rit avec moi
1527 Wild Boys
1528 Oru Nalla Naal Paathu Solren
1529 Home Shopper
1530 The Green Lie
1531 Veerey Ki Wedding
1532 Derren Brown: Pushed to the Edge
1533 Zoo
1534 Love, Chunibyo & Other Delusions! Take On Me
1535 Delirium
1536 Piggy
1537 A Few Less Men
1538 Little Pink House
1539 Hal
1540 The Hunters
1541 Facing the Wind
1542 A Kid Like Jake
1543 Frontier
1544 Exorcist House of Evil
1545 Liz and the Blue Bird
1546 Touch Me Not
1547 Hello & Goodbye
1548 The More You Ignore Me
1549 Kalyanam
1550 Sunday's Illness
1551 Queen
1552 Citizen Jake
1553 Race 3
1554 Deep Blue S

...

28409 Wild Sri Lanka
28410 Atmos
28411 Lost Conquest
28412 Toto
28413 Khanat el-Yak
28414 Et liv på den anden side III: Den hvor Søren Vegger rejser tilbage i tiden
28415 Binhi: The Seed
28416 Pink Boy
28417 M Phase
28418 The Trick With the Gun
28419 We Shall Overcome
28420 Battle Mountain: Graeme Obree's Story
28421 Tamayura: Sotsugyou Shashin Part 1 - Kizashi
28422 The Low Budget Time Machine
28423 Japan: Earth's Enchanted Islands
28424 The Millennials
28425 The Boy who Learned to Fly
28426 The Flesh Of My Lovers
28427 Montedoro
28428 National Hero Lily Vidic
28429 Off
28430 Shuriken Sentai Ninninger The Movie: The Dinosaur Lord's Splendid Ninja Scroll!
28431 Breaker Breaker
28432 Beach Week
28433 Hernia
28434 Howl
28435 The Trials of Spring
28436 World Series of Fighting 17: Shields vs Foster
28437 Wake Up, Leviathan
28438 Finding Phong
28439 Il Bambino di Vetro
28440 Distances
28441 Mimesis
28442 3 Wicketinu 365 Runs
28443 Nightstand
28444 Wide Open Sky
28445 The Story of Us That N

28741 Ratatat - Pitchwork Festival 2015
28742 Family Μember
28743 Alisa in Warland
28744 Sunshine
28745 Call Me Dad
28746 Ratatat - Pitchwork Festival 2015
28747 Tanja Žagar - Concert at 10th anniversery
28748 Voltaire
28749 So What
28750 La Californie
28751 Nomad's Land
28752 So What
28753 La Californie
28754 Nomad's Land
28755 Steve Vai: Rock In Rio 2015
28756 CZW Night of Infamy
28757 The Silent
28758 The Bionic Girl
28759 Bruce Takes Dragon Town
28760 Immortal
28761 Elected
28762 Postpartum
28763 Life Under The Horseshoe
28764 Life Under The Horseshoe
28765 Glory 26: Amsterdam
28766 See You Around
28767 Phantom City
28768 Quero Dizer-te Adeus
28769 It's Complicated
28770 Mommy
28771 Helianthus
28772 Érase una vez en Puerto Rico
28773 Zenaida
28774 Rock in the Red Zone
28775 Extraction
28776 Pete Correale: Let Me Tell Ya
28777 Jinx
28778 It Was a Colossal Teenage Movie Machine: The AIP Story
28779 Shivers Down Your Spine
28780 Billy Idol - Live Austin City Limits Music Festival 2015

29068 Madmast Barkhaa
29069 Still a Rose
29070 Concrete Love - The Böhm Family
29071 Flightless Bird & Merry-Go-Round
29072 Shadow World: The Haunting of Mysti Delane
29073 Saugatuck Cures
29074 Coffee Bloom
29075 RPW: Summer Sizzler 2015
29076 Slimy Piece of Worm-Ridden Filth
29077 Rapid Vengeance
29078 Amok Bamaim
29079 The Taste
29080 Under Your Fingers
29081 This Exists - A Film about Sri Prem Baba
29082 Vanilla
29083 Asia - Axis XXX Live San Francisco MMXII
29084 Captain 3D
29085 Cooking up a Tribute
29086 Paquete #3
29087 Granny, Vanya and the Goat
29088 Why She Cries
29089 Yami Douga 12
29090 The Last Days Of Cinema
29091 Two Girls Descending the Stairs
29092 Akhtar Deadend
29093 Pepi and Muto
29094 Roddenberry's Trek
29095 La France est Notre Patrie
29096 The Down
29097 Modern City
29098 Duality
29099 Caffeinated
29100 Forgiveness
29101 Locked Out
29102 Sueño vagabundo
29103 Northern Great Mountain
29104 Seven Dorms of Death
29105 CAPTUM (lat. Captivity)
29106 Stop
29107 Till D

29389 Fever Room
29390 The Flower & The Bee
29391 The Greater Evil
29392 Awesome Runaway
29393 Unemployment Office
29394 Tokyo Grand Guignol
29395 The Secret Disco Revolution
29396 Live to Live
29397 Family Party
29398 Gordon B. Hinckley: A Giant Among Men
29399 P=ωρ The Dark Energy
29400 The Signalman
29401 A Killer of Men
29402 Henry
29403 The Middle Distance
29404 The Legend of the Palme d'Or
29405 120 Days
29406 White Water
29407 Tell Me Sweet Something
29408 Banglo Berkunci
29409 Her Composition
29410 Do You Dream in Color?
29411 Pikachu and the Pokémon Band
29412 The Stagg Do
29413 Léone, mère & fils
29414 Sankarabharanam
29415 After Spring Comes Fall
29416 Mac Daddy & the Lovers
29417 The Statistical Analysis of Your Failing Relationship
29418 Dogs Without Names
29419 The Farm
29420 Adventures in Comedy
29421 Sleepless Night in Paris
29422 Must Not Think
29423 Miss Bulalacao
29424 Strawberry Shortcake: Berry Best in Show
29425 Oxford Gardens
29426 Manila's Finest
29427 Joanna
29

29726 Hope Bridge
29727 Talakjung vs Tulke
29728 Sky High
29729 Zero Degree
29730 Zen & the Art of Dying
29731 Roommate Wanted - Dead or Alive
29732 The Love Story
29733 Amor
29734 Conducting Boijmans
29735 Bridgebuilders
29736 Sex & Violence with Machspeed
29737 Just Desserts
29738 Real Beleza
29739 Mr Madila
29740 Burden of Peace
29741 Strange Love
29742 Callejero
29743 Alif
29744 Thoppi
29745 Adult Camp
29746 Australien Skies
29747 S.I.D.S.
29748 HADES
29749 L'Aquarium et la Nation
29750 Whiteout
29751 Last Day of School
29752 Check Please
29753 Last Conversation
29754 Under the robes
29755 Muslim Drag Queens
29756 Talking about Food and Other Things
29757 Harry's Bar
29758 The Voice
29759 Lost Memories 2.0
29760 The Sadness of the Trees
29761 Interview with Dr. Axel Stoll. The Movie
29762 Dry
29763 Arcana
29764 Landform 1
29765 Son of the Congo
29766 Simorgh
29767 The Perfect Girl
29768 When Miracle Meets Maths
29769 Cigarette
29770 Onnam Loka Mahayudham
29771 Chaud Lapin
29772 Ano